In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from time import sleep
import requests
import json
import pandas as pd

In [22]:
with open('account.json') as f:
    account = json.load(f);

my_id = account['id']
my_pw = account['password']

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://rankingdak.com/')
driver.find_element_by_xpath('//*[@id="header"]/div[2]/div/ul[1]/li[1]/a').click()
driver.find_element_by_id('login_id').send_keys(my_id)
driver.find_element_by_id('login_pwd').send_keys(my_pw)
driver.find_element_by_class_name('box_btn').click()
sleep(2)
driver.find_element_by_class_name('hm1').click()
driver.find_element_by_class_name('n3').click()
driver.find_element_by_xpath('//*[@id="2322"]/a').click()


select = Select(driver.find_element_by_id('quantity'))
select.select_by_visible_text('300개')
sleep(2)
target_url = driver.current_url

def crawl_login(target_url, filename):
    f = open(filename, 'w', encoding='utf-8')
    page = requests.get(target_url)
    f.write(page.text)
    return page


page = crawl_login(driver.current_url, 'chicken_login.txt')

In [3]:
soup = BeautifulSoup(page.content.decode('utf-8', 'replace'), 'html.parser')
all_tags = soup.find_all(class_='col3')
name_tags = all_tags[0].select('.prd_name')
price_tags = all_tags[0].select('.price')
unit = all_tags[0].select('.price_unit')

In [4]:
chicken_name=[]
chicken_price=[]
for name in name_tags:
    chicken_name.append(name.text.strip())

for price in price_tags:
    chicken_price.append(price.text.strip())

In [5]:
cnt = len(chicken_name)
one_plus_idx=0
weight=[]
dic = {}
for i in range(cnt):
    dic[chicken_name[i]]=chicken_price[i]

for i in range(cnt):    
    if chicken_name[i][-1:] != 'g':
        one_plus_idx = i
        chicken_name[i] = chicken_name[i][:-4] + '100g'

In [6]:
df = pd.DataFrame(chicken_name, columns=['이름'])
df['개수']=10
df.loc[one_plus_idx, '개수']=11 # 10+1 제품 구분해서 개수 11개로 등록 
df['가격']=chicken_price[:120]

In [7]:
df.sort_values(by='가격', ascending=True)
for i in range(cnt):
    df.loc[i, '가격'] = df.loc[i, '가격'][:-1].replace(',', '')

In [20]:
pd.set_option('display.max_rows', None)

df.loc[df['이름']=='수지스 수비드 닭가슴살 550g','개수']=3

for i in range(cnt):
    g = pd.Series(df.loc[i, '이름'][-4:-1])
    g[i] = pd.Series(df.loc[i, '이름'][-4:-1])
    df.loc[i, 'g당 가격'] = int(df.loc[i, '가격']) / (int(df.loc[i, '개수']) * int(g[i][0]))


In [21]:
df.sort_values(by='g당 가격')

,이름,개수,가격,g당 가격
93,수지스 수비드 닭가슴살 550g,3,15500,9.393939
92,수지스 수비드 닭가슴살 550g,3,15500,9.393939
59,햇살닭 페퍼콘 스팀 닭가슴살 200g,10,20850,10.425000
58,햇살닭 페퍼콘 스팀 닭가슴살 200g,10,20850,10.425000
115,닭가슴살한끼 한끼통살 닭가슴살 갈릭 100g,10,11500,11.500000
114,닭가슴살한끼 한끼통살 닭가슴살 갈릭 100g,10,11500,11.500000
113,네이처온 스팀 닭가슴살 오리지널 100g,10,11500,11.500000
112,네이처온 스팀 닭가슴살 오리지널 100g,10,11500,11.500000
117,매일먹닭 촉촉한 닭가슴살 100g,10,12500,12.500000
116,매일먹닭 촉촉한 닭가슴살 100g,10,12500,12.500000


In [19]:
df.loc[df['이름']=='수지스 수비드 닭가슴살 550g','개수']=3